In [0]:
!pip install -q sklearn

In [0]:
%tensorflow_version 2.x

In [0]:
from __future__ import absolute_import , division , print_function , unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf


In [0]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dftest = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

In [0]:
y_train = dftrain.pop('survived')
y_test = dftest.pop('survived')

In [0]:
CATEGORICAL_COLUMNS = ['sex' , 'n_siblings_spouses' , 'parch' , 'class' , 'deck' , 'embark_town' , 'alone']
NUMERIC_COLUMNS = ['age' , 'fare']

feature_column = []

for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_column.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name , vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_column.append(tf.feature_column.numeric_column(feature_name , dtype=tf.float32))

display(feature_column)


In [0]:
def make_input_fn(data_df , label_df , num_epochs=10 ,shuffle=True , batch_size=32):
  def input_funtion():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df) , label_df)) #convert data into 1d data
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs) #repeating epochs to all batches 
    return ds
  return input_funtion

In [0]:
train_input_fn = make_input_fn(dftrain , y_train)
test_input_fn = make_input_fn(dftest , y_test , num_epochs=1 , shuffle=False)

In [0]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_column)
linear_est.train(train_input_fn)
result=linear_est.evaluate(test_input_fn)

In [33]:
print(result['accuracy'])

0.7689394


In [36]:
result = list(linear_est.predict(test_input_fn))
print(result[0]['probabilities'][0])

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp2dcmn_pg/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
0.9334873
